易算法大赛[第二名](https://github.com/analysys/2018_Analysys_2nd_Algorithm_Competition/tree/master/%E6%80%A7%E5%88%AB%E5%B9%B4%E9%BE%84%E9%A2%84%E6%B5%8B)代码复现

由于我用的谷歌colab，他们用的ubuntu,并且在一些指令上存在差别，所以没有完整复现

主要是学习一下他们通过Bag of words 构建 tf-idf 的方法，至于后来10折交叉验证和MLP Classifier，感觉参考价值不大。



```
# 通过BOW模型主要是对设备应用安装列表，

应用的类别信息以及设备数据中应用的使用情况，通过BOW向量来表示。

2.2.TF-IDF

根据BOW向量计算特征的TF-IDF值。
```



In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [3]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import os
os.chdir("drive")
!ls

 aclImdb_v1.tar.gz      practice	        Titanic
 data_description.txt   residual_plot.png       torchtest.ipynb
 drive		        Sales		        train.csv
 GuodongDrive	        sample_submission.csv   Untitled0.ipynb
 House		        Sense1.ipynb	        XGboost_practice1.ipynb
'House price'	        Sense2.ipynb（副本）    XGboost_practice2.ipynb
 Houseprice.ipynb       Sense.ipynb	        Yi
 huawei		        sensePY.ipynb	        剑指offer
 labeledTrainData.tsv   submission.csv	        数组和链表.desktop
'movie review'	        test.csv	        糖尿病数据.txt
 neg		        testData.tsv


In [4]:
os.chdir("Yi")
!ls

device_applist.ipynb


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss
#import LoadData
%matplotlib inline

 设备数据：每个设备上的应用安装列表，设备应用名都进行了hash处理【deviceid_packages.tsv】

In [0]:
device_applist = pd.read_csv('deviceid_packages.tsv',sep='\t',
                             names=['device_id','app_names'])

device_applist['app_names']=device_applist['app_names'].apply(lambda x:x.split(','))
device_applist['app_count']=device_applist['app_names'].apply(lambda x:len(x))

In [7]:
device_applist.head(5)

,device_id,app_names,app_count
0,00009270c4ec26e1d76f5d86847009c9,"[1896072db9ce6406febfc17f681c2086, 90cb852cf34...",3
1,000189ef5d5b951841d416a8c6c5b995,"[1896072db9ce6406febfc17f681c2086, 97d0422a331...",19
2,00026d79a6f0955fc860947724e24765,[c33b35d6254ad9c0c238233eb97a6c60],1
3,0002e3afb8146bc08e40575e45f0eca6,"[1896072db9ce6406febfc17f681c2086, 07e967d75aa...",3
4,0004709a296f9b925ae283efe2f043e7,"[4538778ad75aa8ce61c9d13fb9cb661b, 86f9f299cdb...",16


In [0]:
vectorizer=CountVectorizer()
transformer=TfidfTransformer()
# 所有设备的应用安装列表
apps = device_applist['app_names'].apply(lambda x:' '.join(x)).tolist()
# 设备安装应用稀疏矩阵
cntTf = vectorizer.fit_transform(apps)
# tfidf权重
tfidf=transformer.fit_transform(cntTf)
# TruncateSVD
svd = TruncatedSVD(n_components=550, n_iter=15, random_state=666)
# countvector
app_svd = svd.fit_transform(cntTf)
f_names = ['svd-'+str(x) for x in range(550)]
app_svd = pd.DataFrame(app_svd,columns=f_names)
# add tfidf_sum columns
app_svd['tfidf_sum'] = tfidf.sum(axis=1)
app_svd['device_id'] = device_applist.device_id.values

In [10]:
app_svd

,svd-0,svd-1,svd-2,svd-3,svd-4,svd-5,svd-6,svd-7,svd-8,svd-9,svd-10,svd-11,svd-12,svd-13,svd-14,svd-15,svd-16,svd-17,svd-18,svd-19,svd-20,svd-21,svd-22,svd-23,svd-24,svd-25,svd-26,svd-27,svd-28,svd-29,svd-30,svd-31,svd-32,svd-33,svd-34,svd-35,svd-36,svd-37,svd-38,svd-39,...,svd-512,svd-513,svd-514,svd-515,svd-516,svd-517,svd-518,svd-519,svd-520,svd-521,svd-522,svd-523,svd-524,svd-525,svd-526,svd-527,svd-528,svd-529,svd-530,svd-531,svd-532,svd-533,svd-534,svd-535,svd-536,svd-537,svd-538,svd-539,svd-540,svd-541,svd-542,svd-543,svd-544,svd-545,svd-546,svd-547,svd-548,svd-549,tfidf_sum,device_id
0,0.928765,0.313341,-0.098278,0.505067,-0.115633,0.012930,0.091314,-0.593435,-0.411725,0.150611,-0.503127,-0.370377,-0.068094,0.045181,-0.001173,0.580077,-0.480513,-0.221170,-0.274719,-0.096063,0.085018,-0.038051,-0.066925,-0.102978,-0.192501,0.052284,-0.028985,-0.011048,0.003159,0.072676,-0.009745,-0.005920,0.035138,-0.013443,-0.004088,-0.087387,-0.037158,0.010781,-0.070893,-0.019590,...,0.000396,-0.001791,0.000714,-0.001982,0.000815,0.000332,-0.000316,0.000417,0.000457,-0.000227,0.001076,-0.000299,0.001562,-0.001648,-0.000359,0.000858,-0.000583,0.000432,0.001525,-0.001138,-0.001214,0.000913,-0.001086,-0.000671,-0.001293,0.000336,-0.001231,0.000313,-0.001214,-0.002311,-0.000864,0.001542,0.000144,0.000415,0.001444,0.000956,-0.000179,0.000972,1.645761,00009270c4ec26e1d76f5d86847009c9
1,1.826692,0.804787,-0.143468,1.120996,0.083853,0.182768,0.306204,0.189965,0.524569,0.779197,0.297962,1.053556,-0.098851,0.019069,-0.014399,0.682854,0.191286,0.946073,-0.172613,0.029000,-0.074053,0.931179,0.390231,-0.016640,-0.506261,0.459049,-0.551032,0.271636,0.032233,-0.235279,0.157345,0.673090,0.232426,-0.027179,0.121180,-0.182362,-0.007123,-0.034479,-0.092933,0.136330,...,-0.002643,-0.004420,-0.001209,-0.012117,-0.004059,-0.008577,-0.009581,-0.016864,-0.009261,0.001707,-0.003990,-0.001876,0.001468,-0.012309,-0.006620,0.005406,0.013225,-0.024079,-0.014869,-0.006065,-0.002199,0.019372,0.002947,-0.001035,-0.007787,-0.008146,0.002289,-0.021360,-0.009997,-0.004875,-0.000573,-0.002064,0.017011,-0.011636,0.008813,-0.014623,0.000716,0.002532,3.946912,000189ef5d5b951841d416a8c6c5b995
2,0.001297,-0.000772,-0.000752,0.001064,0.000844,0.001502,0.000898,-0.000230,0.000707,-0.000665,-0.000111,-0.001972,0.000510,-0.000183,-0.001086,0.000274,0.000139,0.000492,-0.001879,0.000524,0.000039,-0.000361,-0.000315,-0.000031,-0.000099,0.000463,0.000273,-0.000006,-0.000445,0.000619,-0.000938,0.000875,-0.001201,-0.001299,-0.001508,-0.000373,-0.001106,-0.000968,0.000959,0.001296,...,-0.118525,-0.056716,-0.081060,-0.010969,0.104281,-0.059609,0.019421,-0.138511,0.001626,-0.097184,-0.003805,-0.024087,-0.068942,-0.013936,0.077285,0.125784,0.005927,0.074324,-0.250812,-0.092378,0.018049,-0.085421,0.053074,0.072110,-0.008074,-0.026868,-0.035493,-0.017925,0.058652,-0.035159,-0.011223,0.036419,-0.022250,0.013606,-0.039114,-0.064160,-0.064936,-0.006899,1.000000,00026d79a6f0955fc860947724e24765
3,1.116279,-0.125085,-0.263445,-0.168328,-0.823002,0.314945,-0.583519,0.201536,-0.048925,0.249408,-0.337425,0.119899,-0.208849,-0.145959,0.391838,-0.034970,-0.025089,0.084701,0.076233,0.000296,-0.065376,0.020706,0.123762,-0.066368,-0.010665,-0.051848,-0.083139,0.040617,-0.022823,0.005235,-0.033106,0.038559,-0.021437,0.004137,-0.023636,0.045126,0.039288,-0.018143,-0.029702,-0.015353,...,-0.000370,0.000334,0.000704,0.000769,-0.000396,-0.000772,0.000372,-0.000511,-0.001419,0.000175,-0.000021,-0.000155,0.000182,-0.001686,-0.001404,-0.000961,-0.000394,0.001766,0.000860,0.000081,-0.000279,-0.000625,-0.001464,-0.000526,0.000032,0.001280,-0.000497,0.000311,-0.001482,-0.001073,0.000500,0.001366,0.000503,0.001189,-0.000909,0.000545,0.000130,-0.000573,1.625475,0002e3afb8146bc08e40575e45f0eca6
4,1.600457,-0.370908,0.389635,-0.012796,-0.583558,0.186859,0.156888,1.004110,-0.213405,-0.496225,0.049091,-0.133461,-0.041525,0.122851,-0.394397,0.100936,-0.067216,0.159604,0.165597,-0.565875,0.242481,-0.519881,-0.068582,0.426

In [0]:
app_tfidf = svd.fit_transform(tfidf)
f_names = ['tfidf'+str(x) for x in range(550)]
app_tfidf = pd.DataFrame(app_tfidf, columns=f_names)

app_tfidf['device_id'] = device_applist.device_id.values

app_svd.to_csv('features/applist_cnt.csv',index=False)
app_tfidf.to_csv('features/applist_tfidf.csv',index=False)

In [0]:
device_applist =  device_applist.merge(app_svd, on='device_id', how='left')

In [16]:
device_applist.head()

,device_id,app_names,app_count,svd-0,svd-1,svd-2,svd-3,svd-4,svd-5,svd-6,svd-7,svd-8,svd-9,svd-10,svd-11,svd-12,svd-13,svd-14,svd-15,svd-16,svd-17,svd-18,svd-19,svd-20,svd-21,svd-22,svd-23,svd-24,svd-25,svd-26,svd-27,svd-28,svd-29,svd-30,svd-31,svd-32,svd-33,svd-34,svd-35,svd-36,...,svd-511,svd-512,svd-513,svd-514,svd-515,svd-516,svd-517,svd-518,svd-519,svd-520,svd-521,svd-522,svd-523,svd-524,svd-525,svd-526,svd-527,svd-528,svd-529,svd-530,svd-531,svd-532,svd-533,svd-534,svd-535,svd-536,svd-537,svd-538,svd-539,svd-540,svd-541,svd-542,svd-543,svd-544,svd-545,svd-546,svd-547,svd-548,svd-549,tfidf_sum
0,00009270c4ec26e1d76f5d86847009c9,"[1896072db9ce6406febfc17f681c2086, 90cb852cf34...",3,0.928765,0.313341,-0.098278,0.505067,-0.115633,0.012930,0.091314,-0.593435,-0.411725,0.150611,-0.503127,-0.370377,-0.068094,0.045181,-0.001173,0.580077,-0.480513,-0.221170,-0.274719,-0.096063,0.085018,-0.038051,-0.066925,-0.102978,-0.192501,0.052284,-0.028985,-0.011048,0.003159,0.072676,-0.009745,-0.005920,0.035138,-0.013443,-0.004088,-0.087387,-0.037158,...,-0.001571,0.000396,-0.001791,0.000714,-0.001982,0.000815,0.000332,-0.000316,0.000417,0.000457,-0.000227,0.001076,-0.000299,0.001562,-0.001648,-0.000359,0.000858,-0.000583,0.000432,0.001525,-0.001138,-0.001214,0.000913,-0.001086,-0.000671,-0.001293,0.000336,-0.001231,0.000313,-0.001214,-0.002311,-0.000864,0.001542,0.000144,0.000415,0.001444,0.000956,-0.000179,0.000972,1.645761
1,000189ef5d5b951841d416a8c6c5b995,"[1896072db9ce6406febfc17f681c2086, 97d0422a331...",19,1.826692,0.804787,-0.143468,1.120996,0.083853,0.182768,0.306204,0.189965,0.524569,0.779197,0.297962,1.053556,-0.098851,0.019069,-0.014399,0.682854,0.191286,0.946073,-0.172613,0.029000,-0.074053,0.931179,0.390231,-0.016640,-0.506261,0.459049,-0.551032,0.271636,0.032233,-0.235279,0.157345,0.673090,0.232426,-0.027179,0.121180,-0.182362,-0.007123,...,-0.002760,-0.002643,-0.004420,-0.001209,-0.012117,-0.004059,-0.008577,-0.009581,-0.016864,-0.009261,0.001707,-0.003990,-0.001876,0.001468,-0.012309,-0.006620,0.005406,0.013225,-0.024079,-0.014869,-0.006065,-0.002199,0.019372,0.002947,-0.001035,-0.007787,-0.008146,0.002289,-0.021360,-0.009997,-0.004875,-0.000573,-0.002064,0.017011,-0.011636,0.008813,-0.014623,0.000716,0.002532,3.946912
2,00026d79a6f0955fc860947724e24765,[c33b35d6254ad9c0c238233eb97a6c60],1,0.001297,-0.000772,-0.000752,0.001064,0.000844,0.001502,0.000898,-0.000230,0.000707,-0.000665,-0.000111,-0.001972,0.000510,-0.000183,-0.001086,0.000274,0.000139,0.000492,-0.001879,0.000524,0.000039,-0.000361,-0.000315,-0.000031,-0.000099,0.000463,0.000273,-0.000006,-0.000445,0.000619,-0.000938,0.000875,-0.001201,-0.001299,-0.001508,-0.000373,-0.001106,...,-0.150277,-0.118525,-0.056716,-0.081060,-0.010969,0.104281,-0.059609,0.019421,-0.138511,0.001626,-0.097184,-0.003805,-0.024087,-0.068942,-0.013936,0.077285,0.125784,0.005927,0.074324,-0.250812,-0.092378,0.018049,-0.085421,0.053074,0.072110,-0.008074,-0.026868,-0.035493,-0.017925,0.058652,-0.035159,-0.011223,0.036419,-0.022250,0.013606,-0.039114,-0.064160,-0.064936,-0.006899,1.000000
3,0002e3afb8146bc08e40575e45f0eca6,"[1896072db9ce6406febfc17f681c2086, 07e967d75aa...",3,1.116279,-0.125085,-0.263445,-0.168328,-0.823002,0.314945,-0.583519,0.201536,-0.048925,0.249408,-0.337425,0.119899,-0.208849,-0.145959,0.391838,-0.034970,-0.025089,0.084701,0.076233,0.000296,-0.065376,0.020706,0.123762,-0.066368,-0.010665,-0.051848,-0.083139,0.040617,-0.022823,0.005235,-0.033106,0.038559,-0.021437,0.004137,-0.023636,0.045126,0.039288,...,-0.001519,-0.000370,0.000334,0.000704,0.000769,-0.000396,-0.000772,0.000372,-0.000511,-0.001419,0.000175,-0.000021,-0.000155,0.000182,-0.001686,-0.001404,-0.000961,-0.000394,0.001766,0.000860,0.000081,-0.000279,-0.000625,-0.001464,-0.000526,0.000032,0.001280,-0.000497,0.000311,-0.001482,-0.001073,0.000500,0.001366,0.000503,0.001189,-0.000909,0.000545,0.000130,-0.000573,1.625475
4,0004709a296f9b925ae283efe2f043e7,"[4538778ad75aa8ce61c9d13fb9cb661b, 86f9f299cdb...",16,1.600

## Train/test

In [32]:
train_path = 'deviceid_train.tsv'
test_path = 'deviceid_test.tsv'
train_data = pd.read_csv('deviceid_train.tsv',header=None,sep='\t', names=['device_id','sex','age'])
test_data = pd.read_csv('deviceid_test.tsv',header=None,names=['device_id'])
train_data.head()                        


,device_id,sex,age
0,bd86d59afa24a839ce6029d718accb19,1,3
1,e7d158c9a8262a35c9cc630a15a9103e,1,5
2,97abdc3828448b5acc7428dd307bc635,2,5
3,e4dbdbf07c9cff03d79f4872e65742b4,1,4
4,6bd4537b2970c5c6ab765c1860b88aa5,1,3


In [33]:
test_data.head()

,device_id
0,2e95e7361f04b5c5c0662e1dcb6fbc65
1,42013074fa7d8d11768e6d3c3e58df56
2,9cc488a211a7c355f99c52ed42e34cb1
3,29dea7b5d58e1e2fe1400a69c5ac33f5
4,365c63569569d88ed44b347fd7c308bd


### Merge(applist)

In [38]:
train_data = train_data.merge(device_applist, on='device_id',how='left')
test_data = test_data.merge(device_applist, on='device_id',how='left')
train_data.head()

,device_id,sex,age,app_names_x,app_count_x,svd-0_x,svd-1_x,svd-2_x,svd-3_x,svd-4_x,svd-5_x,svd-6_x,svd-7_x,svd-8_x,svd-9_x,svd-10_x,svd-11_x,svd-12_x,svd-13_x,svd-14_x,svd-15_x,svd-16_x,svd-17_x,svd-18_x,svd-19_x,svd-20_x,svd-21_x,svd-22_x,svd-23_x,svd-24_x,svd-25_x,svd-26_x,svd-27_x,svd-28_x,svd-29_x,svd-30_x,svd-31_x,svd-32_x,svd-33_x,svd-34_x,...,svd-511_y,svd-512_y,svd-513_y,svd-514_y,svd-515_y,svd-516_y,svd-517_y,svd-518_y,svd-519_y,svd-520_y,svd-521_y,svd-522_y,svd-523_y,svd-524_y,svd-525_y,svd-526_y,svd-527_y,svd-528_y,svd-529_y,svd-530_y,svd-531_y,svd-532_y,svd-533_y,svd-534_y,svd-535_y,svd-536_y,svd-537_y,svd-538_y,svd-539_y,svd-540_y,svd-541_y,svd-542_y,svd-543_y,svd-544_y,svd-545_y,svd-546_y,svd-547_y,svd-548_y,svd-549_y,tfidf_sum_y
0,bd86d59afa24a839ce6029d718accb19,1,3,"[1896072db9ce6406febfc17f681c2086, 07c1809b1fd...",16,1.539203,0.699046,-0.555291,-0.225110,0.228770,-0.338918,-0.738648,0.662865,-0.260018,0.156953,0.178013,0.661034,0.348207,0.155946,0.024194,-0.060073,-0.251609,-0.223364,-0.124596,0.102973,0.083236,0.198122,-0.166004,0.084140,-0.044524,-0.022872,0.056384,-0.058574,-0.075882,-0.024482,-0.054148,-0.031935,-0.092580,0.044032,0.058870,...,-0.045892,-0.131946,0.024120,-0.045378,0.085758,-0.035651,0.053055,-0.023327,-0.001143,-0.034266,-0.024199,0.060620,0.047920,-0.036503,-0.018840,-0.022157,-0.016175,0.046238,-0.092626,-0.023631,0.001324,0.194902,-0.075871,-0.058732,0.028001,-0.010108,-0.055391,0.182253,-0.081677,0.141037,-0.107735,0.060540,-0.082413,0.025754,-0.098498,0.287871,0.069912,-0.246411,-0.250047,3.457704
1,e7d158c9a8262a35c9cc630a15a9103e,1,5,"[e29eb7083bdf54af48352ffa979fc830, 1896072db9c...",12,1.184757,0.735985,-0.064645,0.777342,0.066675,0.009553,-0.644864,-0.250051,0.315026,-0.078636,0.187159,0.502004,0.724971,0.261987,-0.534315,-0.822380,-0.070585,-0.289414,-0.145825,-0.230065,-0.483560,-0.023134,-0.057716,-0.073895,-0.026271,-0.029943,-0.127330,-0.031882,-0.048648,-0.116507,-0.123875,-0.001185,-0.063602,-0.046128,0.010547,...,0.012525,-0.005009,0.017165,0.003481,0.012718,-0.015458,-0.016870,-0.003283,-0.009753,-0.002927,0.011995,-0.002378,-0.002039,0.001161,0.001490,0.001702,0.002036,0.011453,-0.001230,-0.003646,-0.004425,-0.006929,-0.000302,-0.008276,0.004417,0.002653,0.000056,-0.002766,0.011243,-0.005186,0.000797,-0.006243,-0.001576,-0.005125,0.014177,-0.001304,0.008732,0.002561,-0.005910,3.037932
2,97abdc3828448b5acc7428dd307bc635,2,5,"[e92e736fafb3a82c490c60f3491672d4, 1896072db9c...",3,0.759679,0.432890,-0.084837,-0.342851,-0.169882,0.615130,-0.001476,-0.342863,-0.010425,-0.128787,0.060056,-0.107296,-0.080728,-0.019225,-0.078466,-0.026346,0.019509,-0.193624,-0.044066,0.003073,0.163779,-0.046613,-0.030665,-0.008606,-0.120831,0.055265,-0.143949,-0.318150,0.325931,-0.209797,0.092729,-0.153441,0.012837,0.013988,-0.010782,...,-0.003164,-0.003068,-0.000688,-0.005075,0.001717,-0.002979,-0.002433,-0.002969,-0.003282,-0.002035,0.001660,-0.003963,-0.001138,-0.002052,0.001285,0.000324,-0.000508,0.002623,-0.000253,0.000310,0.000468,-0.004414,0.001815,-0.000776,-0.001124,0.001759,-0.001837,-0.001477,-0.000277,-0.001385,-0.002201,0.001254,-0.001974,-0.000630,0.001559,-0.002491,-0.000736,0.001622,-0.000154,1.538901
3,e4dbdbf07c9cff03d79f4872e65742b4,1,4,"[6d0125a3f13a454c98b9841e4ef96b1a, 4b58ecb20fe...",5,0.528722,0.355811,-0.164066,-0.307859,0.551931,-0.594383,0.273638,-0.056354,0.308111,0.239630,0.206303,0.016227,-0.741547,0.551623,-0.294143,-0.235411,-0.262600,0.159090,0.532765,-0.038274,-0.790884,-0.391749,0.436306,-0.713504,0.369333,-0.060410,-0.083898,-0.162242,-0.315118,-0.198010,0.091855,-0.029404,-0.021602,0.072803,-0.022871,...,0.001549,-0.001541,-0.001116,0.001018,0.000685,0.000019,0.000651,-0.003143,-0.000043,-0.001391,0.004310,-0.001118,-0.000978,0.000655,-0.001027,0.001522,0.002035,0.001966,-0.000423,-0.000107,0.000015,0.000634,0.002008,0.000263,0.002667,-0.001543,0.003143,0.002385,0.001992,-0.000505,0.003137,0.001205,0.002059,-0.000887,-0.000531,-0.000935,0.001474,0.000391,-0

# MLPC

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier#多层感知器分类器
from sklearn.metrics import log_loss
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
def train_code(train_data, test_data,label, num_class, n_folds=5):
    labels = train_data[label]
    train_data = train_data.drop(['device_id','sex','age','app_names'],axis=1)
    test_data = test_data.drop(['device_id','app_names'],axis=1)
    train_predvec = np.zeros((train_data.shape[0], num_class))
    test_predvec = np.zeros((test_data.shape[0], num_class))
    SKF = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 2018)
     
    for train_indices, valid_indices in SKF.split(train_data, labels):
        # Training data for the fold
        x_train = train_data.loc[train_indices, :]
        y_train = labels.loc[train_indices, :]
        # Validation data for the fold
        x_valid = train_data.loc[valid_indices, :]
        y_valid = labels.loc[valid_indices, :]
        # MLPC
        mlpc = MLPClassifier(hidden_layer_sizes=(640,640,640),  #300,300,300
                     alpha=0.0033,            
                     batch_size=256,          # 256
                     learning_rate='adaptive',
                     learning_rate_init=0.00054321,
                     random_state=666,
                     verbose=False,
                     early_stopping=True,
                     validation_fraction=0.1)       
        mlpc.fit(x_train, y_train)
         
        train_predvec[valid_indices] = mlpc.predict_proba(x_valid)
        test_predvec += mlpc.predict_proba(test_data)/n_folds
        # Clean up memory
        
        
        
    return train_predvec, test_predvec
   

In [49]:
train_set, test_set = train_code(train_data, test_data,'label', 22, 10)

KeyError: ignored

In [0]:
np.save('new_feature/applist_train.npy',train_set)
np.save('new_feature/applist_test.npy',test_set)